In [1]:
# データ分析用のライブラリ
import pandas as pd
import numpy as np

#前処理用ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# 機械学習モデル関連ライブラリ
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors

# モデル評価関連ライブラリ
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve

In [2]:
# カラムのみが格納されたテーブルを読み込み

df = pd.read_excel('visit_access_log_190217.xlsx')
df

,visit_id,gv_jptop_flg,top_banner_click_flg,gv_bikelineup_flg,gv_bike_contents_flg,gv_bike_detail_flg,gv_welcab_flg,gv_gr_flg,gv_customize_flg,gv_webcatalog_flg,...,buy_used_bike_count,buy_new_bike_flg,buy_used_bike_flg,total_visit_count_before_buy,total_gv_bike_count_before_buy,landing_page_scroll_rate,landing_gv_time,is_buy_flg,is_before_buy_flg,is_after_buy_flg


In [3]:
'''カラム名末尾が「_flg」となっているカラムのみ抽出し、リストに格納する'''
flg_df = df.filter(regex='_flg$').drop(['is_before_buy_flg','is_after_buy_flg'], axis=1)
flg_col_list = flg_df.columns.tolist()

In [4]:
len(flg_col_list)

62

### 乱数を使用したデータフレームを作成

In [5]:
# 乱数を使用して10000×62のデータフレームを作成
np.random.seed(0)

log_df = pd.DataFrame(np.random.randint(low=0, high=2, size=(1000, 62)), columns=flg_col_list)
log_df['visit_id'] = range(len(log_df))

display(log_df.shape, log_df.head())

(1000, 63)

,gv_jptop_flg,top_banner_click_flg,gv_bikelineup_flg,gv_bike_contents_flg,gv_bike_detail_flg,gv_welcab_flg,gv_gr_flg,gv_customize_flg,gv_webcatalog_flg,gv_dop_childseat_flg,...,cv_tradein_simulation_flg,cv_maker_catalog_entry_flg,cv_dealer_catalog_entry_flg,cv_test_drive_flg,cv_talk_reservation_flg,cv_adv_talk_reservation_flg,buy_new_bike_flg,buy_used_bike_flg,is_buy_flg,visit_id
0,0,1,1,0,1,1,1,1,1,1,...,1,1,0,1,0,0,1,1,0,0
1,1,0,1,0,0,0,0,0,1,1,...,0,1,0,1,0,1,1,1,1,1
2,1,0,1,1,1,1,0,1,1,0,...,0,0,0,1,0,1,0,1,0,2
3,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,1,1,1,0,3
4,0,0,1,1,1,0,1,1,1,1,...,0,1,0,0,1,1,1,1,1,4


## 説明変数と目的変数を指定

In [6]:
X = log_df.drop(['visit_id', 'is_buy_flg'], axis=1)
y = log_df['is_buy_flg']

## 学習データとテストデータに分割

In [7]:
# X[train_index]→X.loc[train_index]に変更すればエラーなく実行可能に
# https://stackoverflow.com/questions/51091132/pandas-and-scikit-learn-keyerror-not-in-index
stratifiedKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for train_index, test_index in stratifiedKFold.split(X, y):
    train_X, test_X = X.loc[train_index], X.loc[test_index]
    train_y, test_y = y.loc[train_index], y.loc[test_index]

## パラメータ候補の設定

In [8]:
# 探索したいハイパーパラメータをdict形式で定義

# ロジスティック回帰
lr_param_grid = {
    'C':[1, 10]
}

# ランダムフォレスト
rf_param_grid = {
    'n_estimators':[10, 20],
    'max_features':[5, 10],
    'random_state':[0],
    'max_depth':[5, 10]
}

# SVM
svm_param_grid = {
    'C':[1, 10],
    'gamma':[1, 10]
}

## グリッドサーチ用関数を定義

In [9]:
def create_precision_gridsearchCV(clf, params):
    # n_jobsを-1とすると[WinError32]が発生するので1にしている
    model = GridSearchCV(clf, param_grid=params, scoring='average_precision', cv=5, n_jobs=1)
    
    return model

## グリッドサーチ実行&学習

In [10]:
'''ロジスティック回帰'''
lr_gscv = create_precision_gridsearchCV(LogisticRegression(), lr_param_grid)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
lr_gscv.fit(train_X, train_y)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklea

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1, param_grid={'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='average_precision', verbose=0)

In [11]:
'''ランダムフォレスト'''
# 100万レコード以上の場合、処理に大変時間がかかる（30分程）
rf_gscv = create_precision_gridsearchCV(RandomForestClassifier(), rf_param_grid)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
rf_gscv.fit(train_X, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'n_estimators': [10, 20], 'max_features': [5, 10], 'random_state': [0], 'max_depth': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='average_precision', verbose=0)

In [12]:
'''SVM'''
# 100万レコード以上の場合、処理に大変時間がかかる（240分程）
svm_gscv = create_precision_gridsearchCV(svm.SVC(), svm_param_grid)

# fit関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
svm_gscv.fit(train_X, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'C': [1, 10], 'gamma': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='average_precision', verbose=0)

In [13]:
lr_best_model = lr_gscv.best_estimator_
print(lr_best_model)
print('-'*50)
print(lr_gscv.best_score_) # クロスバリデーション実施時に最も良かったスコア
print('-'*50)
print(lr_gscv.best_params_)# 最も良かったスコアのパラメータの組み合わせ
print('-'*50)
print(lr_gscv.score(test_X, test_y))

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
--------------------------------------------------
0.5044659857352529
--------------------------------------------------
{'C': 10}
--------------------------------------------------
0.475278673448201


In [14]:
rf_best_model = rf_gscv.best_estimator_
print(rf_best_model)
print('-'*50)
print(rf_gscv.best_score_) # 最も良かったスコア
print('-'*50)
print(rf_gscv.best_params_)# 最も良かったスコアのパラメータの組み合わせ
print('-'*50)
print(rf_gscv.score(test_X, test_y))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
--------------------------------------------------
0.5024918229983591
--------------------------------------------------
{'max_depth': 5, 'max_features': 5, 'n_estimators': 20, 'random_state': 0}
--------------------------------------------------
0.54957433384629


In [15]:
svm_best_model = svm_gscv.best_estimator_
print(svm_best_model)
print('-'*50)
print(svm_gscv.best_score_) # 最も良かったスコア
print('-'*50)
print(svm_gscv.best_params_)# 最も良かったスコアのパラメータの組み合わせ
print('-'*50)
print(svm_gscv.score(test_X, test_y))

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=10, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
--------------------------------------------------
0.484394506866417
--------------------------------------------------
{'C': 1, 'gamma': 10}
--------------------------------------------------
0.4824120603015075


#### →ロジスティック回帰のバリデーションスコアが高かったのでロジスティック回帰で予測を行う

## 予測
グリッドサーチによって出された最適なパラメータをテストデータに適用し予測する

In [22]:
# 確率を算出
proba_y = lr_best_model.predict_proba(test_X)
proba_y[0:5]

array([[0.4646966 , 0.5353034 ],
       [0.70449727, 0.29550273],
       [0.60862974, 0.39137026],
       [0.74452944, 0.25547056],
       [0.48190638, 0.51809362]])

In [23]:
pred_y = lr_best_model.predict(test_X)
pred_y[0:5]

array([1, 0, 0, 0, 1])

## 正解率・適合率・再現率・F値を算出

In [19]:
# 混同行列を出力

confusion_matrix(test_y, pred_y)
tn, fp, fn, tp = confusion_matrix(test_y, pred_y).ravel()
# (tp, fn, fp, tn)に順番を変更、それを行列にする
print(np.array([[tp, fn], [fp, tn]]))
print('--------------------------------')
# 二次元配列の場合はnumpy.matrix関数でも表現可能
print(np.matrix([[tp, fn], [fp, tn]]))

[[47 49]
 [51 52]]
--------------------------------
[[47 49]
 [51 52]]


In [20]:
# 小数第2位まで出力
print('ポジネガ判定の正解率は{:.2%}です。'.format(accuracy_score(test_y, pred_y)))
print('ポジネガ判定の適合率は{:.2%}です。'.format(precision_score(test_y, pred_y)))
print('ポジネガ判定の再現率は{:.2%}です。'.format(recall_score(test_y, pred_y)))
print('ポジネガ判定のF値は{:.2%}です。'.format(f1_score(test_y, pred_y)))

ポジネガ判定の正解率は49.75%です。
ポジネガ判定の適合率は47.96%です。
ポジネガ判定の再現率は48.96%です。
ポジネガ判定のF値は48.45%です。


In [21]:
# 適合率・検出率・F値をまとめて表示するsklearn.metrics.classification_report
# 出力部分の「0」「1」は今回の場合「未購入」「購入」という意味、「support」は「各ラベルのデータ数」を意味する
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.51      0.50      0.51       103
           1       0.48      0.49      0.48        96

   micro avg       0.50      0.50      0.50       199
   macro avg       0.50      0.50      0.50       199
weighted avg       0.50      0.50      0.50       199



参考URL

https://qiita.com/tomov3/items/039d4271ed30490edf7b

https://github.com/hwpwk/NLP/blob/master/RandomForest%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E7%AB%A0%E3%81%AE%E3%83%9D%E3%82%B8%E3%83%8D%E3%82%AC%E5%88%A4%E5%AE%9A%E3%81%A8%E9%87%8D%E8%A6%81%E5%BA%A6%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%A8graphviz%E3%81%A7%E6%9C%A8%E6%A7%8B%E9%80%A0%E5%8F%AF%E8%A6%96%E5%8C%96%26%E3%81%9D%E3%81%AE%E6%AD%A3%E8%A7%A3%E7%8E%87%E3%83%BB%E9%81%A9%E5%90%88%E7%8E%87%E3%83%BB%E5%86%8D%E7%8F%BE%E7%8E%87%E3%83%BBF%E5%80%A4_ver_train_test_split(%E6%96%87%E6%9B%B8%E3%81%AE%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E5%8C%96(bag_of_words)%2B%E3%83%9D%E3%82%B8%E3%83%8D%E3%82%AC%E6%95%99%E5%B8%AB%E3%83%A9%E3%83%99%E3%83%AB%E8%BF%BD%E5%8A%A0%2BDataFrame%E5%8C%96)_181107.ipynb